In [2]:
from scipy.optimize import minimize
import pprint
import qsharp
from qsharp.chemistry import load_broombridge, load_fermion_hamiltonian, load_input_state, encode

print("Done")

Adding package microsoft.quantum.chemistry.jupyter.

Done


In [15]:
# Loading molecular data from Broombridge
filename = 'data/hydrogen_0.2.yaml'
broombridge_data =  load_broombridge(filename)

# Retrieve basis set and geometry used to generate the input data
basis_set = broombridge_data.problem_description[0].basis_set
geometry = broombridge_data.problem_description[0].geometry

# Retrieve the nuclear repulsion and the one-electron integrals
nuclear_repulsion = broombridge_data.problem_description[0].coulomb_repulsion['Value']
one_electron_integrals =  broombridge_data.problem_description[0].hamiltonian['OneElectronIntegrals']['Values']

# print("nuclear_repulsion = ", nuclear_repulsion)
# print("one_electron_integrals = ", one_electron_integrals, "\n\n")

# Loading the Fermion Hamiltonian
fermion_hamiltonian = broombridge_data.problem_description[0].load_fermion_hamiltonian()
# print("fermion_hamiltonian :: \n", fermion_hamiltonian, "\n\n")
# print(dir(fermion_hamiltonian))

# Loading the input state ansatz
ansatz = load_input_state(filename, "UCCSD |G>")
# print("ansatz :: \n", ansatz, "\n\n")
# print(dir(ansatz))

# Encoding the Hamiltonian using Jordan-Wigner encoding into the Hartree-Fock state
jw_hamiltonian = encode(fermion_hamiltonian, ansatz)
pp = pprint.PrettyPrinter(indent=4)
print("jw_hamiltonian :: \n")
pp.pprint(jw_hamiltonian)

jw_hamiltonian :: 

(   4,
    (   [   ([0], [0.17120128499999998]),
            ([1], [0.17120128499999998]),
            ([2], [-0.222796536]),
            ([3], [-0.222796536])],
        [   ([0, 1], [0.1686232915]),
            ([0, 2], [0.12054614575]),
            ([0, 3], [0.16586802525]),
            ([1, 2], [0.16586802525]),
            ([1, 3], [0.12054614575]),
            ([2, 3], [0.1743495025])],
        [],
        [([0, 1, 2, 3], [0.0, -0.0453218795, 0.0, 0.0453218795])]),
    (   3,
        [   ((0.001, 0.0), [2, 0]),
            ((-0.001, 0.0), [3, 1]),
            ((0.113, 0.0), [2, 3, 1, 0]),
            ((1.0, 0.0), [0, 1])]),
    -0.09883444600000002)


In [4]:
"""
The Jordan-Wigner encoding of the Hamiltonian is a tuple of the form (a1, a2, in_state, a4), where:
	a1 is the constant term of the Hamiltonian
	a2 is the coefficient of the Pauli Z operator
	in_state is the input state is a tuple of the form (b1, amplitudes), where:
		b1 is the constant term of the input state
		amplitudes is a list of tuples of the form ((var_param, 0.0), coefficient), where:
			var_param is the variational parameter
			coefficient is the coefficient of the Pauli Z operator
	a4 is the coefficient of the Pauli X operator

This method returns the list of variational parameters of the Jordan-Wigner encoding of the Hamiltonian.
"""
def get_var_params(jw_hamiltonian):
	_, _, i, _ = jw_hamiltonian
	_, amplitudes = i
	params = [param for ((param, _), _) in amplitudes]
	return params[:-1]

var_params = get_var_params(jw_hamiltonian)
print("var_params ::", var_params)

var_params :: [0.001, -0.001, 0.113]


In [5]:
# Estimate the energy of the Hamiltonian
"""
The H2 ground state energy at equilibrium, R=1.40 bohr, is 1.174476 hartree.
"""
estimate = qsharp.QSharpCallable("Microsoft.Quantum.Chemistry.JordanWigner.VQE.EstimateEnergy", "")
estimated_energy = estimate.simulate(jwHamiltonian = jw_hamiltonian, nSamples=1e18)
print("Energy evaluated at {0} : {1}".format(var_params, estimated_energy))

Energy evaluated at [0.001, -0.001, 0.113] : -1.0560238743040393


In [6]:
"""
The Jordan-Wigner encoding of the Hamiltonian is a tuple of the form (a1, a2, in_state, a4), where:
	a1 is the constant term of the Hamiltonian
	a2 is the coefficient of the Pauli Z operator
	in_state is the input state is a tuple of the form (b1, amplitudes), where:
		b1 is the constant term of the input state
		amplitudes is a list of tuples of the form ((var_param, 0.0), coefficient), where:
			var_param is the variational parameter
			coefficient is the coefficient of the Pauli Z operator
	a4 is the coefficient of the Pauli X operator

This method sets the variational parameters of the Jordan-Wigner encoding of the Hamiltonian, to be used in the energy evaluation and optimization.
"""
def set_var_params(input_params, jw):
	a1, a2, in_state, a4 = jw
	b1, amplitudes = in_state
	new_amplitudes = [((input_params[i], 0.0), amplitudes[i][1]) for i in range(len(input_params))]
	new_amplitudes.append(amplitudes[-1])

	in_state = (b1, new_amplitudes)
	jw = (a1, a2, in_state, a4)
	return jw

"""
This method evaluates the energy of the Hamiltonian, given the variational parameters.
"""
def energy_evaluation(var_parameters, jw, n_samples):
	# var_parameters = list(var_parameters)
	jw = set_var_params(var_parameters, jw)
	energy = estimate.simulate(jwHamiltonian = jw, nSamples=n_samples)
	print("Energy evaluated at {0} : {1}\n".format(var_parameters, energy))
	return energy

In [9]:
def VQE(in_var_params, jw, n_samples):
	# Minimize the energy of the Hamiltonian
	result = minimize(energy_evaluation, in_var_params, args=(jw, n_samples), method='COBYLA', tol=1e-9, options={'disp': True, 'maxiter': 200,'rhobeg' : 0.05})
	return result

result = VQE(var_params, jw_hamiltonian, 1e18)
print("result ::", result)

Energy evaluated at [ 0.001 -0.001  0.113] : -1.0560238743038477

Energy evaluated at [ 0.051 -0.001  0.113] : -1.0542461847004285

Energy evaluated at [0.001 0.049 0.113] : -1.0542801923068277

Energy evaluated at [ 0.001 -0.001  0.163] : -1.0171240626453892
Energy evaluated at [-0.00128029 -0.00323667  0.06310213] : -1.087587247959818

Energy evaluated at [-0.00409604 -0.00599855  0.01325794] : -1.1115432421007063

Energy evaluated at [-0.02204644 -0.02388511 -0.02984489] : -1.1249464890131518

Energy evaluated at [ 0.01650616 -0.03460251 -0.05982547] : -1.1318479015442628
Energy evaluated at [ 0.01208354  0.00163777 -0.09398806] : -1.1365568574611777

Energy evaluated at [ 0.00409545 -0.01821874 -0.13917555] : -1.1359150668001825

Energy evaluated at [ 0.05322308  0.02612499 -0.10840729] : -1.1339504311759128
Energy evaluated at [-0.00726559  0.01740781 -0.09537279] : -1.1365379338421255

Energy evaluated at [ 0.01978569  0.01062591 -0.09800555] : -1.1364210576308125

Energy evaluat

In [10]:
fci_value = -1.1372704220924401
print(result.fun)
print("Difference with exact FCI value :: ", abs(result.fun - fci_value))

-1.1372704139924368
Difference with exact FCI value ::  8.100003334732264e-09
